In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Hello, I am sharing a new solution example on Medical Cost Personnel Dataset in this worksheet. Your comments on the subject are important to me.

Some parts may be lack of opinion or indirectly, I am open to all new suggestions, I hope this case study will be useful for you.


1. General Information About Data


Medical Cost Personnel data consists of various information about individuals and corresponding downloads. Our goal on this data is to be able to estimate the loads with the information given.


Throughout the study, we will first load the data and make rough checks. We will then examine the variables, look at the links between them and the target variable, and then strive to find various effective variables

2. First Look at Data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_csv("/kaggle/input/insurance/insurance.csv")

In [ ]:
data.head()

In [ ]:
data.isnull().sum()/len(data)

In [ ]:
data.describe().T

To learn more about the data, it is very useful to look at the correlation of variables with each other, and a simple method for this is the heatmap in a nice graph.

In [ ]:
corr=data.corr()
sns.heatmap(corr)

It seems that age is a little more descriptive than other variables. Let's draw various graphs to understand it a little more.

In [ ]:
sns.pairplot(data,hue="smoker")

In the graph above, the target variable seems to have a close relationship with age, bmi and smoking. For example, it becomes clear that smokers pay more. Or it is quite clear that smokers pay higher wages for people with the same bmi value.

In [ ]:
sns.scatterplot(x=data.bmi,y=data.charges,hue=data.smoker)

We have said body mass index for BM. If we break down this variable into categorical data, we can look at and interpret, for example, people who smoke sigra with a normal bmi value, non-smokers with a normal bmi value, or more.

Adding New Variables


I will divide the bmi values ​​by the values ​​I found from the bmi table. These,

* 18.5 ve altı underweight
* 18.5 ve 25 arası normal
* 25 ve 30 arası overweight
* 30 ve 35 arası moderately obese
* 35 ve 40 arası several obese
* 40 ve üzeri very several obese


In [ ]:
a=0
bmi_grup=[]

while a<1339:
    
    if (data.bmi[a]<=18.5):
        bmi_grup.append("under_weight")
    elif (data.bmi[a]>18.5 and data.bmi[a]<=25):
        bmi_grup.append("normal")
    elif (data.bmi[a]>25 and data.bmi[a]<=30):
        bmi_grup.append("over_weight")
    elif (data.bmi[a]>30 and data.bmi[a]<=35):
        bmi_grup.append("moderately_obese")
    elif (data.bmi[a]>35 and data.bmi[a]<=40):
        bmi_grup.append("several_obese")
    else:
        bmi_grup.append("very_obese")
        
    a+=1
    


In [ ]:
bmi_grup=pd.DataFrame(bmi_grup,columns=["bmi_grup"])


In [ ]:
data["bmi_grup"]=bmi_grup

In [ ]:
data.head()

We have added our groups, now let's look at the graphic.

In [ ]:
sns.scatterplot(x=data.bmi_grup,y=data.charges,hue=data.smoker)

For example, people with moderately_obese value have a net wage difference compared to smoking. The same comment can be made in severa_obese and very_obese.

Now, I want to distinguish smokers and non-smokers in these three groups. For this, I will use a structure similar to the above.


In [ ]:
i=0
obese_smoker=[]

while i<1339:
    
    if(data.bmi_grup[i]=="very_obese" or data.bmi_grup[i]=="several_obese" or data.bmi_grup[i]=="moderately_obese") and (data.smoker[i]=="yes"):
        obese_smoker.append(1)
    else:
        obese_smoker.append(0)
    
    i+=1

In [ ]:
obese_smoker=pd.DataFrame(obese_smoker,columns=["obese_smoker?"])
data["obese_smoker?"]=obese_smoker

In [ ]:
data.head()

In [ ]:
sns.scatterplot(x=data.charges,y=data["obese_smoker?"])

Even if some situations overlap between 3000 and 4000, we still have separated those who pay high wages and those who do not.

Now let's divide the variable of age just as we divide the bmi values. So again, let's try to find out if there are those who are the same age but different bmi groups or pay a difference according to smoking.

* 18-20 arası young
* 20-30 arası young adult
* 30-40 arası adult
* 40-50 arası old
* 50 üzeri very old


I guess we did a little bit of injustice to people over 50  ¯\_(ツ)_/¯

In [ ]:
u=0
age_segment=[]

while u<1339:
    
    if(data.age[u]<=20):
        age_segment.append("young")
    elif(data.age[u]>20 and data.age[u]<=30):
        age_segment.append("young_adult")
    elif(data.age[u]>30 and data.age[u]<=40):
        age_segment.append("adult")
    elif(data.age[u]>40 and data.age[u]<=50):
        age_segment.append("old")
    else:
        age_segment.append("very_old")
        
    u+=1

In [ ]:
age_segment=pd.DataFrame(age_segment,columns=["age_segment"])
data["age_segment"]=age_segment
data.head()

In [ ]:
sns.countplot(data.age_segment)

Okay, we didn't divide ages too disproportionately.

In [ ]:
sns.scatterplot(x=data.bmi,y=data.charges,hue=data.smoker)


In [ ]:
sns.scatterplot(x=data.bmi,y=data.charges,hue=data.age_segment)

There is something like this between the two graphs above: When the BMI value rises above 30, suddenly high differences start to occur between age groups (2nd chart) The part with these high wages corresponds to the smokers in the first graph, so we can create a new variable from here.

I want to make this new variable by taking bmi_groups whose bmi value is greater than 30, smokers and old / very_old ones. By the way, there will not be many examples that fit this situation, but I think it would be useful to add.

In [ ]:
p=0
obese_smoker_oldies=[]

while p<1339:
    
    if(data.bmi_grup[p]=="moderately_obese" or data.bmi_grup[p]=="several_obese" or data.bmi_grup[p]=="very_obese") and (data.smoker[p]=="yes") and (data.age_segment[p]=="old" or data.age_segment[p]=="very_old"):
        obese_smoker_oldies.append(1)
    else:
        obese_smoker_oldies.append(0)
    
    p+=1

In [ ]:
obese_smoker_oldies=pd.DataFrame(obese_smoker_oldies,columns=["obese_smoker_oldies"])

In [ ]:
data["obese_smoker_oldies"]=obese_smoker_oldies
data.head()

In [ ]:
sns.scatterplot(x=data.obese_smoker_oldies,y=data.charges)

In [ ]:
data.dtypes

In [ ]:
cols=["sex","smoker","region","bmi_grup","age_segment"]

from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

for i in cols:
    
    data[i]=le.fit_transform(data[i])
    
data.head()

In [ ]:
x=data.iloc[:,0:6]
x1=data.iloc[:,7:11]
x=pd.concat([x,x1],axis=1)
y=data.iloc[:,6:7]
x.head()


In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=0)




4. **XGBOOST**

In [ ]:
from xgboost import XGBRegressor

xgb=XGBRegressor()
xgb.fit(x_train,y_train)
xgb_tahmin=xgb.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score


print("R^2 skoru",r2_score(y_test,xgb_tahmin),
     "Hata Kareler Ortalaması", np.sqrt(mean_squared_error(y_test,xgb_tahmin)))

In [ ]:
data.describe()

Let's try the tune operation now.

In [ ]:
from sklearn.model_selection import GridSearchCV

xgb_params={
    
    "colsample_bytree": np.arange(0.2,1,0.1),
    "max_depth": np.arange(3,10,1),
    "learning_rate": np.arange(0.1,1,0.1)
    
}


xgb_tune=XGBRegressor()
xgb_cv=GridSearchCV(xgb_tune,xgb_params,cv=10,n_jobs=-1,verbose=2)

xgb_cv.fit(x_train,y_train)

In [ ]:
xgb_cv.best_params_


I hope my first example in Kaggle has been useful for you.

See you again.

Alper Temel.

You can support the study by giving an upvote :)

Thanks !